### <center>1. Importing the dataset</center>

In [1]:
# Importing the dataset
import pandas as pd
df = pd.read_csv('icc_world_cup_2019.csv')
df

,team1,team2,winner,margin,ground,venue
0,England,South Africa,England,104 runs,Kennington Oval,London
1,West Indies,Pakistan,West Indies,7 wkts,Trent Bridge,Nottingham
2,New Zealand,Sri Lanka,New Zealand,10 wkts,Sophia Gardens,Cardiff
3,Afghanistan,Australia,Australia,7 wkts,County Ground,Bristol
4,South Africa,Bangladesh,Bangladesh,21 runs,Kennington Oval,London
5,England,Pakistan,Pakistan,14 runs,Trent Bridge,Nottingham
6,Afghanistan,Sri Lanka,Sri Lanka,34 runs,Sophia Gardens,Cardiff
7,South Africa,India,India,6 wkts,The Rose Bowl,Southampton
8,Bangladesh,New Zealand,New Zealand,2 wkts,Kennington Oval,London
9,Australia,West Indies,Australia,15 runs,Trent Bridge,Nottingham


In [2]:
# Checking the presence of null values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   team1   48 non-null     object
 1   team2   48 non-null     object
 2   winner  48 non-null     object
 3   margin  48 non-null     object
 4   ground  48 non-null     object
 5   venue   48 non-null     object
dtypes: object(6)
memory usage: 2.4+ KB


In [3]:
# Check duplicacy of the values
df.count()

team1     48
team2     48
winner    48
margin    48
ground    48
venue     48
dtype: int64

### <center>2. Calculation & Aggregation of Data</center>

In [4]:
# Calculating the total matches played by each team
team1_total = df.groupby('team1').size()
team2_total = df.groupby('team2').size()
total_matches_played = pd.concat([team1_total, team2_total]).groupby(level=0).sum()
total_matches_played

Afghanistan      9
Australia       10
Bangladesh       9
England         11
India           10
New Zealand     11
Pakistan         9
South Africa     9
Sri Lanka        9
West Indies      9
dtype: int64

In [5]:
# Calculate the 'wins' for each team
team1_wins = df[df['team1'] == df['winner']].groupby('team1').size()
team2_wins = df[df['team2'] == df['winner']].groupby('team2').size()
total_wins = pd.concat([team1_wins, team2_wins]).groupby(level=0).sum()
total_wins

Australia       7
Bangladesh      3
England         8
India           7
New Zealand     6
Pakistan        5
South Africa    3
Sri Lanka       3
West Indies     2
dtype: int64

In [6]:
# Calculate the 'nr' (No Result) for each team
team1_nr = df[df['winner'] == 'NR'].groupby('team1').size()
team2_nr = df[df['winner'] == 'NR'].groupby('team2').size()
total_nr = pd.concat([team1_nr, team2_nr]).groupby(level=0).sum()
total_nr

Bangladesh      1
India           1
New Zealand     1
Pakistan        1
South Africa    1
Sri Lanka       2
West Indies     1
dtype: int64

In [7]:
# Merge the calculation series into a single Pandas dataframe
all_matches = pd.concat([total_matches_played, total_wins, total_nr], axis=1)
all_matches.columns = ['total_matches', 'total_wins', 'total_nr']
all_matches = all_matches.fillna(0)
all_matches = all_matches.astype(int)
all_matches

,total_matches,total_wins,total_nr
Afghanistan,9,0,0
Australia,10,7,0
Bangladesh,9,3,1
England,11,8,0
India,10,7,1
New Zealand,11,6,1
Pakistan,9,5,1
South Africa,9,3,1
Sri Lanka,9,3,2
West Indies,9,2,1


In [8]:
# Calculating the losses and the points.
all_matches['loss'] = all_matches['total_matches'] - all_matches['total_wins'] - all_matches['total_nr']
all_matches['pts'] = (all_matches['total_wins'] * 2) + all_matches['total_nr']
all_matches

,total_matches,total_wins,total_nr,loss,pts
Afghanistan,9,0,0,9,0
Australia,10,7,0,3,14
Bangladesh,9,3,1,5,7
England,11,8,0,3,16
India,10,7,1,2,15
New Zealand,11,6,1,4,13
Pakistan,9,5,1,3,11
South Africa,9,3,1,5,7
Sri Lanka,9,3,2,4,8
West Indies,9,2,1,6,5


In [9]:
# Sort the DataFrame by 'pts' in descending order
all_matches = all_matches.sort_values(by=['pts'], ascending=False)
all_matches

,total_matches,total_wins,total_nr,loss,pts
England,11,8,0,3,16
India,10,7,1,2,15
Australia,10,7,0,3,14
New Zealand,11,6,1,4,13
Pakistan,9,5,1,3,11
Sri Lanka,9,3,2,4,8
Bangladesh,9,3,1,5,7
South Africa,9,3,1,5,7
West Indies,9,2,1,6,5
Afghanistan,9,0,0,9,0
